### Nomenclature 
NN      Neural Network  
PINN       Physics-Informed Neural 
___

# Background 
The goal of today's experiment is to train a neural network (NN) and a physics-informed neural network (PINN) to predict the trajectory of a projectile motion. We'll be exploring the influence of hyperparameters and the number of colocation points on the training, and comparing the NN with the PINN.
Throughout the experiment, we'll be applying machine learning theory practically and familiarizing ourselves with the PyTorch library. The experiment focuses on a simple physical system to emphasize the machine learning aspects.

# Workflow
This Notebook is a tutorial designed for hands-on learning. Please follow the chronological order unless advised otherwise.  
You will implement a neural network (NN) and a physics-informed neural network (PINN) and complete various execution tasks.  
Good luck!
___
# Chronological order
## NN implementation and execution
### Implementation:
First you will implement a NN. This includes the following steps:
- importing and preparing data.
- creating the NN architecture.
- defining the data- and total loss functions.
- writing the training procedure.


### Execution tasks:
After implementing the code, it is necessary to test its functionality. Proceed to the following execution tasks:
1. __Verification__:  
    Execute training and visualize the results.
2. __Hyperparameter__:  
    Test the impact of hyperparameters on model optimization.

## PINN implementation and execution
### Implementation:
Implement a physics-informed neural network (PINN) using the steps outlined below:
- Implement the function `compute_physics_informed_loss`
- Include the Physics Loss in `compute_total_loss`

### Execution tasks:
The work is finalized with the following two execution tasks:

3. __NN vs. PINN__:  
    Compare the performance of PINN and regular NN on different datasets.  
4. __Collocation points__:  
    Identify the effect of the collocation points when training a PINN.

# Implementation NN Part 1
Import all necessary libraries and classes by running the following cell.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import trange

from utils.physics_data_generator import ProjectileDataGenerator, PhysicalSystem
from utils.Animator import AnimationGenerator, SolutionVisualizer

## Data generation

The data we are working with describes the trajectory of a projectile that is slowed down by a drag force.
It is generated using two imported classes.  
- The `physical_system` class contains all relevant information about the projectile and its motion.  
- To create labeled data, we use the `data_generator`, which produces time values and corresponding position targets.

In [ ]:
# Use existing code to generate training data
physical_system = PhysicalSystem()
data_generator = ProjectileDataGenerator(physical_system)

# Generate training data
data_generator.integrate()

# Get training data
time = data_generator.time
position = data_generator.position

# Get the physical system parameters
g = physical_system.g
Cd = physical_system.drag_coeff
A = physical_system.cross_area
m = physical_system.mass
mu = physical_system.coeff


## Data visualization

To better understand the data, visualize the projectile's trajectory. Half of the necessary code has already been provided.  
To create the plot, replace the '...' with the correct code.  
The plot should include the following properties:
- Plot the x-position on the x-axis using the first column of the position tensor.
- Plot the y-position on the y-axis using the second column of the position tensor.

__Hint:__  
First, determine the shape of the position tensor, and then use slicing to access the appropriate dimensions.  
Learn how to select coloums via slicing [here](https://note.nkmk.me/en/python-numpy-ndarray-slice/)


In [ ]:
plt.figure(figsize=(8, 4))
#plt.plot(...)
plt.title(rf"Projectile Motion with Drag $\mu$ = {physical_system.drag_coeff:.2f}")
plt.xlabel("x [m]")
plt.ylabel("y [m]")
plt.legend()

## Data preparation
This section covers data preparation for training.  
Data can be used for training and testing. To obtain accurate testing results, it is crucial to avoid using the same data for testing as for training. The dataset is therefore split into two complementary sets.

Therefore, the `prepare_data` function indexes each value of the data set. Then, it selects an evenly distributed amount of 'num_train' indices for training purposes and uses the rest for testing:

| `idx_train` | `all_idx` | `idx_test` |
|:---------:|:---------:|:---------:|
| 1 | &larr; 1 |  |
|  | 2 &rarr; | 2 |
|  | 3 &rarr; | 3 |
| 4 | &larr; 4 |  |
|  | 5 &rarr; | 5 |
| ... | ... | ... |

(This is just an example.)

The data is then saved in dictionaries `train_ds` and `test_ds`. `time_vals` are used as inputs and `position_vals` as targets.

Implement this logic by completing the code below.

In [ ]:
def prepare_data(
        time_vals : torch.Tensor, 
        position_vals : torch.Tensor, 
        num_train : int = 20,  
        max_idx : int = len(time)-1,
        num_t_col : int = 100,
    ):
    """
    Create training data for training procedure.

    This function generates training data using a subset
    of time values ('time_vals' tensor) and corresponding 
    x- and y-components ('position_vals' tensor).
    Therefore the following steps need to be done:
    

    Step 1: Data Point Selection
            To split the data into training and testing data, 
            three Index tensors need to be created.

            The 'all_idx' tensor includes indices from 0 to the dataset's length.
            The 'idx_train' tensor includes 'num_train' evenly spaced indices for training.
            The 'idx_test' tensor includes the remaining indices not in 'idx_train'.

            - Create a tensor 'all_idx' containing indices between 0 
              and 'len(time_vals)' to select data points.
            - Create a tensor 'idx_train' containing 'num_train'evenly spaced indices 
              between 0 and max_idx to select data points.
            - Generate a mask with booleans of len(all_idx) that gives False statements 
              for entries that are both in 'all_idx' and in 'idx_train'. (Just inspect the code)
              (see [torch.isin](https://pytorch.org/docs/stable/generated/torch.isin.html)))
            - Build a tensor 'idx_test' by applying the mask on 'all_idx'. (Just inspect the code)
            - Ensure that the data type of all idx tensors are integer.

    Step 2: Training Data Creation
            - Define 't_train', 'x_train', and 'y_train' tensors 
              by selecting index values 'idx_train' from 'time_vals' and 'position_vals'.
            - We don't want to make changes to the original data 
              therefore use '.clone().detach()'.
            - The neural network expects a tensors with each value in a separate row.
              Format the tensor size as 'torch.Size([num_train, 1])'
              by applying the .unsqueeze(1) method.
        
    Step 3: Testing Data Creation
            Repeat the procedure of step to by selecting index values 'idx_test'.
    
    Step 4: Colocation points
            For the physics loss, we need evenly spaced time values 
            between 0 and the last time value.
            - Using the torch.linspace() method, build 't_col', 
              a tensor that includes 'num_t_col' evenly spaced values 
              between 0 and the last value in 'time_vals'.
              (see [torch.linspace](https://pytorch.org/docs/stable/generated/torch.linspace.html))
            - Use the .unsqueeze(1) method.
            - To declare the tensor as a trainable parameter,
              Ensure that the tensor requires gradient.
              (see [torch.Tensor.requires_grad](https://pytorch.org/docs/stable/generated/torch.Tensor.requires_grad.html)

    Step 5: Dataset Creation
            Create datasets train_ds and test_ds by using dictionaries 
            containing "inputs", "targets_x", "targets_y" and "t_col" entries.
        
    Step 6:  Return the created datasets.
    
    Parameters
    ----------
    time_vals : torch.Tensor
        A tensor containing time values.
    position_vals : torch.Tensor
        A tensor containing x- and y-components of the displacement vector.
    num_train : int, optional
        The number of data points to select from the provided labeled dataset. Default is 20.
    max_idx : float, optional
        The maximum index value to select from data. Default is len(time)-1.
    num_t_col : int, optional
        The number of evenly spaced time values used as colocation points for the physics loss.
        Default is 100.
        
    Returns
    -------
    Dict
        A dictionary containing "inputs", "targets_x", "targets_y", 
        and "t_col" entries for training data.
    Dict
        A dictionary containing "inputs", "targets_x", "targets_y", 
        and "t_col" entries for testing data.
    """
    
    # Step 1: Create a tensors of indices that correspond to the selected time values.
    #all_idx = 
    #idx_train = 
    mask = ~torch.isin(all_idx, idx_train)
    idx_test = all_idx[mask]

    # Assertion checks: Does assertion statements to ensure the correctness of your implementation.
    assert len(idx_train) + len(idx_test) == len(all_idx), "The sum of training and testing indices should be equal to the total number of indices."
    assert len(idx_train) == num_train, "The number of training indices should be equal to the specified number of training data points."
    assert idx_train[-1] == max_idx, "The last index of the training indices should be equal to the maximum index value."
    assert all_idx.dtype == torch.int, "The data type of the indices should be integer."
    assert idx_train.dtype == torch.int, "The data type of the training indices should be integer."
    
    # Step 2: Get 't_train', 'x_train' and 'y_train' by selecting values from the provided data.
    #t_train = time_vals[idx_train].clone().detach().unsqueeze(1)
    #x_train = 
    #y_train = 

    # Assertion check
    assert t_train.shape == x_train.shape == y_train.shape == torch.Size([num_train, 1]), "The shapes of t_train, x_train, and y_train should be torch.Size([num_train, 1])."
    
    
    # Step 3: Get 't_test', 'x_test', 'y_test' from the provided data
    #t_test = 
    #x_test = 
    #y_test = 

    # Assertion check
    assert t_test.shape == x_test.shape == y_test.shape == torch.Size([len(time_vals)-num_train, 1]), "The shapes of t_test, x_test, and y_test should be torch.Size([len(time_vals)-num_train, 1])."


    # Step 4: Generate a t_col tensor.
    # t_col = 

    # Assertion check
    assert t_col.shape == torch.Size([num_t_col, 1]), "The shape of t_col should be torch.Size([num_t_col, 1])."
        
    
    # Step 5: Create datasets 'train_ds' and 'test_ds' by using dictionaries 
    #train_ds = 
    #test_ds = 

    # Assertion check
    assert len(train_ds) == len(test_ds) == 4, "The datasets should contain four entries." 

    # Step 6: return datasets
    pass # replace the pass statement with the return statement

__Checkpoint__:  
In the following cell, run the function. If you have implemented everything correctly, the assert statements should not produce any errors.

In [ ]:
train_ds, test_ds = prepare_data(time, position)

## Model creation

In [ ]:
class PINN(nn.Module):
    """
    Physics-Informed Neural Network (PINN) Class
    
    This class as a subclass of nn.Module defines the architecture of the PINN model. 
    It is designed to use differential equations while incorporating physics-based constraints.
    The process consists of the following steps:
    
    Step 1: Model Initialization (Just inspect the code)
        - Initialize the PINN model as a subclass of nn.Module.

    Step 2: Constructor Definition (Just inspect the code)
        - Build a constructor to configure the model's architecture.
        - Utilize the nn.Linear class from the PyTorch library 
          for defining layers and connections. 

    Step 3: Forward Pass Mechanism
        - Define the forward pass mechanism for the model, 
          where input data flows through the layers
          to produce predicted outputs.
       
    """
    def __init__(self):
        """
        Constructor for the PINN class.
        
        Initializes with the xavier_uniform function the layers of the neural network:
        (see [Xavier_uniform](https://pytorch.org/cppdocs/api/function_namespacetorch_1_1nn_1_1init_1ace282f75916a862c9678343dfd4d5ffe.html))])
        (see [nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html))
        - Input layer fc1 taking a tensor with time data.
        - One hidden fully connected layer fc2 with 64 neurons.
        - Output layer fc_x for predicting the x-coordinate.
        - Output layer fc_y for predicting the y-coordinate.
        
        Parameters
        ----------
        None
            
        Attributes
        ----------
        fc1 : nn.Linear
            First fully connected layer.
        fc2 : nn.Linear
            Second fully connected layer.
        fc_x : nn.Linear
            Output layer for x-coordinate prediction.
        fc_y : nn.Linear
            Output layer for y-coordinate prediction.
        
        Returns
        -------
        None
        
        """
        super(PINN, self).__init__()
        
        # Step 2: Configure model architecture as described earlier
        torch.manual_seed(42)  # Set seed for reproducibility
        self.fc1 = nn.Linear(1, 64)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.fc2 = nn.Linear(64, 64)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        self.fc_x = nn.Linear(64, 1)
        torch.nn.init.xavier_uniform_(self.fc_x.weight)
        self.fc_y = nn.Linear(64, 1)
        torch.nn.init.xavier_uniform_(self.fc_y.weight)
    
    def forward(self, t):
        """
        Perform a forward pass through the PINN model.
        
        This method defines the forward pass mechanism of the PINN model, where
        the input data t is processed through the layers to produce predicted
        outputs for both x-coordinate (x_output) and y-coordinate (y_output).
        The following steps are necessary:
        
    1. First Fully Connected Layer with GELU Activation:
        - Pass the input tensor 't' through the 'fc1' linear layer.
        - Apply the GELU activation function 'torch.nn.functional.gelu(...)' to the output.

    2. Second Fully Connected Layer with GELU Activation:
        - Pass the output of the previous step ('t') through the 'fc2' linear layer.
        - Apply the GELU activation function to the output.

    3. Output Layer for Predicted Coordinates:
        - Compute the predicted x-coordinate by passing the transformed tensor 't' 
          through the 'fc_x' layer.
        - Compute the predicted y-coordinate by passing the same transformed tensor 't' 
          through the 'fc_y' layer.
        
        Parameters
        ----------
        t : torch.Tensor
            Input data tensor.
            
        Returns
        -------
        x_output : torch.Tensor
            Predicted x-coordinate.
        y_output : torch.Tensor
            Predicted y-coordinate.
        """

        # Step 1: Apply the first fully connected layer with GELU activation
        t = torch.nn.functional.gelu(self.fc1(t))
        
        # Step 2: Apply the second fully connected layer with GELU activation
        #t =
        
        # Step 3: Produce predicted x and y coordinates using output layers and return them.
        #x_output =   # Predicted x-coordinate
        #y_output =   # Predicted y-coordinate
        
        pass #replace the pass with the return of the predicted x and y coordinates



## Data loss

In [ ]:
def compute_data_loss(model : nn.Module, dataset: dict):
    """
    Define the data loss for the PINN model.
    
    This function calculates the loss based on the discrepancy between the predicted
    and actual data points, typically used for data-driven training of the PINN. The
    following steps are involved:
    

    Step 1: Model Prediction
        - Compute the predicted x and y values by calling the neural network model.
          on the "inputs" entry of the dataset.

    Step 2: Loss Calculation
        - Calculate the mean squared error loss for both the x and y components using
        the 'nn.MSELoss' class.
        (see [MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html))

    Step 3: Loss Combination
        - Return the combined data loss as the sum of 'loss_x' and 'loss_y'.
    
    Parameters
    ----------
    model : nn.Module
        The physics-informed neural network model.
    dataset : dict
        A dictionary containing "inputs", "targets_x", "targets_y" and "t_col" entries.
        
    Returns
    -------
    torch.Tensor
        The combined data loss based on predicted vs. actual data.
    """
    # Step 1: Compute predicted x and y values using the neural network model.
    #         Replace ... with the appropriate code.
    #x_pred, y_pred = model(...)
    
    # Step 2: Calculate mean squared error loss
    mse_loss = nn.MSELoss()
    #loss_x = mse_loss(x_pred, dataset["targets_x"])
    #loss_y = 

    # Assert section: Ensures the correct shapes and values (explanation in the next cell)
    if model == Dummy_NN:
        assert x_pred.shape == (20, 1), "Incorrect shape for x_pred"
        assert y_pred.shape == (20, 1), "Incorrect shape for y_pred"
        assert round(x_pred[0,0].item(), 6) == -0.066156, "Incorrect value for x_pred"
        assert round(y_pred[0,0].item(), 6) == -0.310818, "Incorrect value for y_pred"
        assert round(loss_x.item(), 6) == 560.922668, "Incorrect value for loss_x"
        assert round(loss_y.item(), 6) == 209.860199, "Incorrect value for loss_y"
    
    # Step 3: Return the combined (x and y) data loss
    pass # replace the pass statement with the return statement

__Checkpoint:__  
In the next cell, you will test your defined ``compute_data_loss`` function on a generated dataset ``dummy_train_ds`` and an instance ``Dummy_NN`` of the PINN.  
If implemented correctly, the prediction/ and loss values/ and shapes pruduced inside the ``compute_data_loss`` function will always be the same, regardless of the randomly generated network parameters.
This is due to the predefined seed value used for parameter initialization, which ensures consistent distribution every time.  

If you run the following cell and no error occurs, it means that your `compute_data_loss` and the `PINN` class have been implemented correctly.

In [ ]:
Dummy_NN = PINN()

dummy_train_ds, dummy_test_ds = prepare_data(time, position)

dummy_data_loss = compute_data_loss(Dummy_NN, dummy_train_ds)

# Implementation PINN 


## Physicis informed loss

If you come here for the first time and you have just implemented the `compute_data_loss` function,  
leave the the `compute_physics_informed_loss` function as it is. 

If you come here for the second time, complete the `compute_physics_informed_loss` function implementation.  
You will need some physics background information:
___

### Physics background

This PINN aims to learn and predict the trajectory of a projectile slowed down by a drag force.

The magnitude of the force depends on the following parameters:
1. $\rho$ the Air density $(\text{kg}/\text{m}^3)$, 
2. $C_d$ the drag coefficient, 
3. $A$ the Cross-sectional area $(\text{m}^2)$

All these parameters can be summarized to one coefficient:
- $\mu = 0.5 * \rho * C_d * A$

Moreover, the drag force depends on the square of the projectile's speed:
- $\vec F = - \mu * \vec v * |\vec v|$ 
where $\vec F$ can be split up in $x$-and $y$-components:
- $F_x = - \mu * \frac{dx}{dt} * |\vec v|$ and 
- $F_y = - \mu * \frac{dy}{dt} * |\vec v|$

The resulting differential equations describe the path of the projectile:
- $\frac{d^2x}{dt^2} = F_x/m$
- $\frac{d^2y}{dt^2} = F_y/m - g$ 

Where $m$ is the Mass of the projectile $(\text{kg})$ and $g$ the earths gravitational acceleration. 


In [ ]:
def compute_physics_informed_loss(model : nn.Module, dataset: dict):
    """
    Define the physics-informed loss for the PINN model.
    
    This function calculates the loss used to incorporate the underlying physics
    principles into the PINN. The following steps are involved:
    
    Step 1: Data Preparation
            - Unpack 't_col' from the dataset.

    Step 2: Model Prediction
            - Predict x and y values by calling the neural network model on 't_col'.

    Step 3: Gradient Computation
            - Compute the first gradients 'dx_dt' and 'dy_dt', and second gradients 
            'd2x_dt2' and 'd2y_dt2', using the 'torch.autograd.grad' method.
            (see[grad](https://pytorch.org/docs/stable/generated/torch.autograd.grad.html))

    Step 4: Velocity and Drag Calculation
            - Calculate the speed 'v' using the Euclidean norm of the vector [dx_dt, dy_dt].
            - Define the x and y components of the drag force.

    Step 5: Loss Calculation
            - Calculate the mean squared error loss for both the x and y components using
            the 'nn.MSELoss' class.
            (see [MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html))

    Step 6: Loss Combination
            - Return the combined physics-informed loss as the sum of 'loss_x' and 'loss_y'.
    
    Parameters
    ----------
    model : nn.Module
        The physics-informed neural network model.
    dataset : dict
        A dictionary containing "inputs", "targets_x", "targets_y" and "t_col" entries.
        
    Returns
    -------
    torch.Tensor
        The combined (x and y) physics-informed loss.
    """
    # Step 1: Unpack t_col from the dataset
    #t_col = 
    
    # Step 2: Predict x and y values using the neural network model
    #x_pred, y_pred = 
    
    # Step 3: Compute first and second gradients
    #dx_dt = torch.autograd.grad(x_pred, t_col, grad_outputs=torch.ones_like(x_pred), create_graph=True)[0]
    #d2x_dt2 = 
    
    #dy_dt = 
    #d2y_dt2 = 

    # Assertion checks
    if model == Dummy_NN:
        assert round(dy_dt[0,0].item(), 6) == 0.010456 , "Incorrect value for dy_dt, check the gradient computation"
        assert round(d2y_dt2[0,0].item(), 6) == -0.011869 , "Incorrect value for d2y_dt2, check the gradient computation"
        assert round(d2x_dt2[0,0].item(), 6) == -0.025199 , "Incorrect value for d2x_dt2, check the gradient computation"
    
    # Step 4: Calculate the speed v using the square root (torch.sqrt()) of the sum 
    #         of squares of dx_dt and dy_dt and define x and y component of the drag force.
    #v = 
    #Fx = -mu 
    #Fy = 
    
    # Step 5: Compute the mean squared error loss 
    #         and store by defining variables called 'loss_x' and 'loss_y'.
    #mse_loss = 
    #loss_x = 
    #loss_y = 

    # Step 6: Return the combined (sum of x and y) physics-informed loss
    pass


__Checkpoint:__  
If you run the following cell and get a value of __96.1587__, it means that your `compute_physics_informed_loss` function has been implemented correctly.

In [ ]:
dummy_physics_informed_loss = compute_physics_informed_loss(Dummy_NN, dummy_train_ds)

print("Dummy physics informed Loss:",dummy_physics_informed_loss)

# Implementation NN Part 2

## Total Loss

In [ ]:
def compute_total_loss(model : nn.Module, dataset: dict, activate_physics=False):
    """
    Define the total loss for the physics-informed neural network.

    This function computes the total loss for the PINN model by combining two
    different components: data loss and physics-informed loss.
    The following steps are involved:

    Step 1: Data Loss Calculation
        - Determine the data loss using the 'compute_data_loss' function.

    Step 2: Physics-Informed Loss Calculation (if already implemented). If not proceed to Step 3)
        - If 'activate_physics' is set to True, calculate the physics-informed loss
        using the 'compute_physics_informed_loss' function.
        - Else, set the physics loss to zero.

    Step 3: Total Loss Combination
        - Return the combined total loss 
          as the sum of 'data_loss' and 'physics_loss' (if already implemented).

    Parameters
    ----------
    model : nn.Module
        The physics-informed neural network model.
    dataset : dict
        A dictionary containing "inputs", "targets_x", "targets_y", and "t_col" entries.
    activate_physics : bool, optional
        Whether to activate the physics-informed loss. Default is False.

    Returns
    -------
    torch.Tensor
        The combined total loss considering data and physics constraints.
    """
    # Step 1: Calculate the data loss
    #data_loss = 

    # Step 2: Calculate the physics-informed loss if 'activate_physics' is True


    # Step 3: Combine data loss and physics loss (if activated) and return the total loss
    
    pass # replace the pass statement with the return statement


__Checkpoint:__  
If you run the following cell and no error occurs, it means that your `compute_total_loss` function has been implemented correctly.

In [ ]:
dummy_total_loss = compute_total_loss(Dummy_NN, dummy_train_ds)

print("Dummy total Loss:",dummy_total_loss)

## Executive function

In [ ]:
def execute(
        model : nn.Module,
        train_ds: dict,
        test_ds: dict,
        lr : float = 0.01, 
        num_epochs : int = 300,
        activate_physics : bool = False,
    ):
    """
    Execute the training procedure for a physics-informed neural network model.

    This function trains the model using specified hyperparameters and returns relevant data.
    The process involves the following steps:
    
    Step 1: Optimizer Initialization
            The optimizer is used to update the model's parameters during training. Therefore:
            - Define 'params' by calling the parameters() method on the model.
              (see [PARAMETERS](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters))
            - Configure the Adam optimizer with a specified learning rate 'lr' and the 'params'.
              More informations on the Adam optimizer can be found here:
              (see [ADAM](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html))


    Inside the training loop:

        Step 2: Loss Calculation
                Calculate the 'train_loss' and 'test_loss' 
                using the 'compute_total_loss' function.
                Make sure, 'activate_physics' is set to 'activate_physics'.




    Parameters
    ----------
    model : nn.Module
        The neural network model to be trained and evaluated.
    train_ds : dict
        A dictionary containing training data, including inputs and targets.
    test_ds : dict
        A dictionary containing test data, including inputs and targets.
    lr : float, optional
        Learning rate for the optimizer. Default is 0.01.
    num_epochs : int, optional
        Number of training epochs. Default is 300.
    activate_physics : bool, optional
        Whether to activate the physics-informed loss. Default is False.

    Returns
    -------
    Dict
        A dictionary containing the following entries:
        - "train_loss_evolution": A list of training loss values.
        - "test_loss_evolution": A list of test loss values.
        - "predictions_list": A list of model predictions.
        - "train_targets_x": A tensor of x-coordinates of the training targets.
        - "train_targets_y": A tensor of y-coordinates of the training targets.
        - "test_targets_x": A tensor of x-coordinates of the test targets.
        - "test_targets_y": A tensor of y-coordinates of the test targets.
        if activate_physics:
        - "x_pred_col": A tensor of x-coordinates of the colocation points.
        - "y_pred_col": A tensor of y-coordinates of the colocation points.

    """    
    
    # Step 1: Configure optimizer 
    # params = 
    optimizer = optim.Adam(...)  # Replace the '...' with the model parameters and the learning rate.
    
    # Initialize the dictionary and lists
    predictions_dict = {}
    train_loss_evolution = []
    test_loss_evolution = []
    predictions_list = []
    
    # Define Loading Bar
    loading_bar = trange(1, num_epochs + 1)
    
    # Set up the training loop for the specified number of epochs ('num_epochs')
    for epoch in loading_bar:

        # Step 3: Compute the total loss for the training and the test data.
        #train_loss = 
        #test_loss = 
        
        # Calculate the training loss ('train_loss') gradients using backpropagation, 
        # then update the model parameters 
        # and set the gradients to zero.
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        # Append the current loss value in the loss lists
        train_loss_evolution.append(float(train_loss))
        test_loss_evolution.append(float(test_loss))
           
        # Compute the model prediction and store it in the respective list.
        predictions = model(test_ds['inputs'])
        predictions_list.append(predictions)

        # Print current epoch and losses
        loading_bar.set_description(f"Epoch: {epoch}")
        loading_bar.set_postfix({"Test Loss": test_loss.item(), "Train Loss": train_loss.item()})

    # Fill the dictionary as specified and return it.
    predictions_dict["train_loss_evolution"] = train_loss_evolution
    predictions_dict["test_loss_evolution"] = test_loss_evolution
    predictions_dict["predictions_list"] = predictions_list
    predictions_dict["train_targets_x"] = train_ds["targets_x"]
    predictions_dict["train_targets_y"] = train_ds["targets_y"]
    predictions_dict["test_targets_x"] = test_ds["targets_x"]
    predictions_dict["test_targets_y"] = test_ds["targets_y"]

    if activate_physics:
        x_pred_col, y_pred_col = model(test_ds["t_col"])
        predictions_dict["x_pred_col"] = x_pred_col
        predictions_dict["y_pred_col"] = y_pred_col


    return predictions_dict

# Execution NN

## Task 1: Verification
Execute Training and Visualize 
    Execute the training for the following set of (hyper)parameters.  
    Plot the loss evolution during training and the final prediction of the model (after training).
    Compare the final predictions to the true solution of the trajectory. 
___
### Datapreparation
- Use default parameters for the data generation
- Use __20 points__ (linearly spaced in time) from the generated data for the `train_ds`.  

### Training execution
- Learning rate $\eta = 0.005$
- Training epochs $n_\text{epochs} = 300$
- Set ``physics_activated`` = `False`
___



In [ ]:
# Step 1: Initialize the model by calling the PINN class created earlier.

# Step 2: Generate training and test datasets by calling the 'prepare_data' function.

# Step 3: Execute the training and testing of the model.


## Visualization
In the upcoming execution tasks, you will need to visualize various model predictions.  
To facilitate this process, you will use the SolutionVisualizer class.  
### Task
To understand how this class works, add the appropriate  
code inside the `_customize_first_subplot` and `_customize_second_subplot` methods where indicated by (...).

In [ ]:
Visualizer = SolutionVisualizer(predictions_dict)
Visualizer.plot_solution(title=rf'Projectile Motion with Drag $\mu$ = {physical_system.drag_coeff:.2f}') # Add title

# Animation
Congratulations! You have successfully trained the model. Now it's time to relax and watch a movie.  
Please run the following cell to see the model training in action.  


In [ ]:
# Example usage:
animator = AnimationGenerator(predictions_dict,
                              train_ds, 
                              test_ds)
animator.create_animation('lets_see.mp4', frames=len(predictions_dict["train_loss_evolution"]), fps=30)

## Task 2: Hyperparameters 
Finding the best configuration for a given task by tuning all hyperparameters of  
the model, such as learning rate, model depth, and model width, can be a tedious process.  
We already did some of that work for you, but you will now have the opportunity to test the impact of the learning rate.

To roughly assess that, adjust the learning rate and observe the model's performance.
1. Create the necessary datasets and implement a loop for the following list of learning rates.

    Learning rates = [1, 1e-1, 1e-2, 1e-3, 1e-4]

   Train the neural network in each loop using the configuration from task 1, except for the learning rate.  
   Append the `prediction_dict`'s in a list called `pred_list`.

   __Hint__:  
   Don't forget to initialize a new ``model`` in each loop.

2. Utilize the `SolutionVisualizer` class to compare the loss evolutions and final predictions in two plots.
This will be done automatically by executing the cell below.
3. Which learning rate yields the best results?  
Explain why. 

In [ ]:
# Initialize a dict that can store the prediction and loss data
pred_list = []

# Define the Looplist 'Learning_rates'
Learning_rates = [1, 1e-1, 1e-2, 1e-3, 1e-4]

# Create Training and Test datasets
...

# Loop through the Learning_rates and train the model 
...


In [ ]:
for i, sub_dict in enumerate(pred_list):
    Visualizer = SolutionVisualizer(sub_dict)
    Visualizer.plot_solution(title=rf'Learning rate = {Learning_rates[i]}',
                             filename=f"lr_data_{i}.svg")


# PINN
Congratulations on completing all tasks related to the NN.  
To proceed with the next Execution Task (cell below), you must first unlock the PINN functionalities.
- Implement the `compute_physics_informed_loss` function [higher up in the code.](#physics-background) (You may need to scroll)

# Execution PINN

## Task 3: NN vs. PINN
The purpose of this task is to compare the performance of a regular NN and a PINN.  
The comparison will be made by training both networks on labeled datasets with the following configurations:

- 20 equally spaced training points over the entire data set.
- 10 equally spaced training points over the first half of the data. 
- 2 training points using the first and last point of the data set.  

1. For each of these data sets, execute one training of a PINN (with physics loss) and one training of regular NN (without physics loss) using the parameter configuration from task 1 (except the training data points). 
Moreover, we use the default values of:
    - Learning rate $\eta = 0.005$
    - Training epochs $n_\text{epochs} = 500$


**Your task** is to write the executive code to run and save the training for each configuration using the following steps:
- Use the `prepare_data` function to create the datasets.
- Train the PINN/NN using the `train_model` function.
- Save the `prediction_dict`s in a list called `pred_list`.
The location in the code where you should implement the steps is indicated by `(...)`.

2. Compare the results using the `SolutionVisualizer`.  
This will be done automatically. 
Your task is to describe and interpret the results in the report:
- What differences do you observe between the NN and PINN predictions?
- How does the number of training points affect the predictions?
- How does the PINN perform compared to the NN?
- Is the comparison fair?

In [ ]:
# List of tuples with dataset keys and boolean values for activating physics during training
data_list = [
    ("data_0", False),  
    ("data_0", True), 
    ("data_1", False), 
    ("data_1", True), 
    ("data_2", False), 
    ("data_2", True)
]
config_dict = {
    "data_0": (20, len(time)-1), # Containing (num_train, max_idx)
    "data_1": (10, len(time)//2), 
    "data_2": (2, len(time)-1)
}

# Initialize a list to store predictions that will be the result of the training
pred_list = []

In [ ]:
# Execute the training for the given configurations

for data, physics in data_list:
    # Print the name of the current dataset
    print(f"Training on dataset {data} with physics={physics}")
    print(f"Using {config_dict[data][0]} training points and a maximum index of {config_dict[data][1]}")

    # Prepare the data for the current dataset
    train_ds, test_ds = ...

    # Initialize the model by calling the PINN() class created earlier.
    model = ...

    # Execute the training for the given data_set
    predictions_dict = ...

    # Append the predictions to the pred_list
    ...
    

In [ ]:
# Visualize the results

for i, sub_dict in enumerate(pred_list):
    Visualizer = SolutionVisualizer(sub_dict)
    Visualizer.plot_solution(title=rf'Physics activated = {data_list[i][1]}',
                             filename=f"NN_PINN_data_{i}.svg")


## Task 4: Collocation points
The goal is to determine how collocation points affect the training of a PINN.  
To achieve this, we train the PINN using the follwing configuration and adjust the number of collocation points:
- 2 training points using the first and last point of the data set.
- Learning rate $\eta = 0.005$
- Training epochs $n_\text{epochs} = 500$

We will test the following number of collocation points:
- 2 linearly spaced over time
- 5 linearly spaced over time
- 20 linearly spaced over time

1. Implement the training for each number of collocation points using the following steps.
You can organize the code similar to the previous task.

2. Plot and compare the loss evolution and final prediction of all four configurations. 

3. What can you see, and how can you explain the behavior?

In [ ]:
# Initialize an empty dictionary to store data
data_dict = {}
pred_list = []

num_collocation_points = [2, 5, 20]

In [ ]:
# Loop and execute the training for different collocation points

...

In [ ]:
for i, sub_dict in enumerate(pred_list):
    Visualizer = SolutionVisualizer(sub_dict)
    Visualizer.plot_solution(title=rf'Number of colocation points = {num_collocation_points[i][0]}',
                             filename=f"col_data_{i}")